# Download the trained model *StaffUNet2* and test image

Here we download the required files hosted on Google Drive using `gdown` command.

In [ ]:
# Get model
!gdown 1KfsaSoGQOjtUo_A2_J-GZ8F8rAxWvbpX
# Get test image
!gdown 1hF36KTun8wHNAi8DMzgexGm8SgyWilI6

# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import cv2

# Utility functions

For this model, `jaccard_index` function was used for computing metrics during the training. When loading the model we require to input also the requirements, so this function is defined here as well.

`get_model` is a utility function to load the model given its path.

In [ ]:
smooth = 1e-12

def jaccard_index(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)

def get_model(model_path):
    dependencies = {'jaccard_index': jaccard_index}
    model = keras.models.load_model(model_path, custom_objects=dependencies)
    return model


## Show image

Simple function for showing images using `matplotlib` library.

In [ ]:
def show_image(image):
  _, axs = plt.subplots()
  axs.imshow(image, cmap='gray')
  axs.axis('off')
  plt.show()

# Core

Our model is trained on 512x512 images so, in order to use it on variable sized sheet music images, we need another method.

## Chunks

This approach is based on dividing the image in **chunks**: pieces of the image of size 512x512. If the size of the image is not multiple of this chunk size, we make a white border of right and bottom side of the image.
The two functions defined here do the following:


*   `divide_in_chunks` takes an image and returns a list of chunks of size `chunk_size` plus some smaller edge chunks that are smaller (if width and height aren't multiple of `chunk_size`
*   `reassemble_image` takes a list of chunks and return the reassembled image



In [ ]:
def divide_in_chunks(image, chunk_size=512):
    img_chunks = []
    for x in range(0, image.shape[1], chunk_size):
        for y in range(0, image.shape[0], chunk_size):
            y2 = min(y + chunk_size, image.shape[0])
            x2 = min(x + chunk_size, image.shape[1])
            img_chunk = image[y:y2, x:x2]
            img_chunks.append(img_chunk)

    return img_chunks

def reassemble_image(chunks, width, height, chunk_size):
    rows = (height // chunk_size)
    if height % chunk_size != 0:
       rows += 1

    assembled_image = np.empty((height, 0, 3), dtype=np.uint8)
    i = 0
    while i < len(chunks):
        column = chunks[i]
        j = i + 1
        while j % rows != 0:
            column = np.concatenate((column, chunks[j]), axis=0)
            j += 1
        assembled_image = np.hstack((assembled_image, column))
        i += rows

    return assembled_image

## Chunk normalization

Each chunk is a piece of image, but this could still not be in the right format. For example, for how the model was trained, the symbols of the image must **white on black background**, so we need to perform a `bitwise_not`.

During the test of the model, it has been tried to add borders to chunks as *safety borders*, to see if the produced image was better "understood" by the model. The option has been kept available.

This function has been developed to work also with chunks of sizes different from `chunk_size`. If we adjust image size as previously described, however, there is no need to check for chunk sizes. This option was made available during previous version of this notebook, and it still is. So, if we submit a chunk of different sizes, this function adds borders to the dimensions to make the image square and then of the correct size.

In [ ]:
def normalize_image(image, safety_border=0, chunk_size=512):
    img = image.copy()

    if len(image.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    border_v, border_h = (0, 0)
    if img.shape[0] < chunk_size <= img.shape[1]:
        img = img[:, :chunk_size]
        border_v = int((chunk_size - img.shape[0]) / 2) + 1
    elif img.shape[1] < chunk_size <= img.shape[0]:
        img = img[:chunk_size, :]
        border_h = int((chunk_size - img.shape[1]) / 2) + 1
    elif img.shape[0] < chunk_size and img.shape[1] < chunk_size:
        border_v = int((chunk_size - img.shape[0]) / 2) + 1
        border_h = int((chunk_size - img.shape[1]) / 2) + 1

    if border_v != 0 or border_h != 0:
      img = cv2.copyMakeBorder(img, border_v, border_v, border_h, border_h, cv2.BORDER_CONSTANT, value=[255, 255, 255])
      img = img[:chunk_size, :chunk_size]
    if safety_border != 0:
      img = cv2.copyMakeBorder(img, safety_border, safety_border, safety_border, safety_border,
                             cv2.BORDER_CONSTANT, value=[255, 255, 255])
      img = cv2.resize(img, (chunk_size, chunk_size))

    img = cv2.bitwise_not(img)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img, border_h, border_v


## Staff Remover

The following class represents the main tool to perform Staff line removal. The only method meant to be used from outside is `remove_staffs`, the others are used internally. The only required parameter is `images`, that is a list of chunks.

It first performs the previously described normalization, and then uses the loaded model `StaffUNet2`. After that, the obtained mask is fist dilated vertically to better cover the staff lines, and then it's applied on the chunk (the mask is used to delete the related pixel).

The processed chunks with staffs removed are returned.

In [ ]:
class StaffRemover:
    def __init__(self, model_path):
        self.model = get_model(model_path)
        self.safety_border = 0
        self.input_images, borders_h, borders_v, images, pred_bin = None, None, None, None, None

    def prepare_chunks(self):
        input_images = []
        borders_h = []
        borders_v = []
        for image in self.images:
          input_image, border_h, border_v = normalize_image(image, self.safety_border)
          input_images.append(input_image)
          borders_h.append(border_h)
          borders_v.append(border_v)

        input_images = np.array(input_images)
        input_images = (input_images / 255)
        input_images = input_images.astype(np.float32)
        return input_images, borders_h, borders_v

    def process_result(self, index):
      output = np.squeeze(self.pred_bin[index], axis=2)
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))

      output_converted = output.astype(np.uint8)
      dilated = cv2.morphologyEx(output_converted, cv2.MORPH_DILATE, kernel)
      mask = dilated == 1

      black_color = np.zeros(shape=(512, 512, 3))

      # Apply the mask to retain BGR values where the mask is True
      result = ((np.where(mask[..., None], black_color, [self.input_images[index]])[0]) * 255).astype(np.uint8)
      _, result = cv2.threshold(result, 50, 255, cv2.THRESH_BINARY)
      result = cv2.bitwise_not(result)
      # Remove the safety border
      result = cv2.resize(result, (512 + self.safety_border*2, 512 + self.safety_border*2))
      if self.safety_border != 0:
        result = result[self.safety_border:-self.safety_border, self.safety_border:-self.safety_border, :]

      def show_chunk_progress(index):
          _, axs = plt.subplots(1, 3)
          axs[0].imshow(self.images[index], cmap='gray')
          axs[0].axis('off')
          axs[1].imshow(output_converted, cmap='gray')
          axs[1].axis('off')
          axs[2].imshow(result, cmap='gray')
          axs[2].axis('off')
          plt.show()

      # show_chunk_progress(i)
      return result


    def remove_staffs(self, images, thresh=0.6, safety_border=0):
      self.images = images
      self.safety_border = safety_border
      self.input_images, borders_h, borders_v = self.prepare_chunks()

      preds = self.model.predict(self.input_images)
      optimal_bin_thr = thresh
      self.pred_bin = (preds >= optimal_bin_thr).astype(np.int64)

      results = []
      for i in range(self.pred_bin.shape[0]):
        result = self.process_result(i)
        h, w, _ = result.shape

        results.append(result[max(borders_v[i]-1, 0):min(h-borders_v[i]+1, h), max(borders_h[i]-1, 0):min(w-borders_h[i]+1, w)])

      self.images = None
      self.safety_border = None
      self.input_images = None
      self.safety_border = 0
      self.pred_bin = None
      return results


## Image size adjustment

Instead of changing every chunk by adding the correct border, we change the size of the entire image to be multiple of `chunk_size` adding a white border.

In [ ]:
def adjust_image_size(image, chunk_size=512):
  h, w = image.shape[0], image.shape[1]
  borders = [0, 0]
  if h % chunk_size != 0:
    new_h = chunk_size * ((h // chunk_size ) + 1)
    borders[0] = new_h - h
  if w % chunk_size != 0:
    new_w = chunk_size * ((w // chunk_size ) + 1)
    borders[1] = new_w - w

  return cv2.copyMakeBorder(image, 0, borders[0], 0, borders[1], cv2.BORDER_CONSTANT, value=[255, 255, 255])

# Usage

We are ready to use our staff remover on sheet music images

## Loading of the image and chunks
We first read the image using the OpenCV library, we adjust its dimensions and then we divide it into chunks using the previous functions.

In [ ]:
image = cv2.imread("/content/test_staff.png", cv2.IMREAD_COLOR)
h, w = image.shape[0], image.shape[1]
adjusted = adjust_image_size(image)
new_h, new_w = adjusted.shape[0], adjusted.shape[1]
chunks = divide_in_chunks(adjusted)

## Staff remover usage

We use the StaffRemover class, first passing the model path and then calling the `remove_staffs` on the list of chunks.

In [ ]:
remover = StaffRemover("/content/StaffUNet2.keras")

without_staff = remover.remove_staffs(chunks)

## Final result

Once we have the list of chunks with staff lines removed, we can use the function `reassemble_image` to rebuild the original image but without staffs.

In [ ]:
symbols_image = reassemble_image(without_staff, new_w, new_h, 512)[:h, :w, :]
show_image(symbols_image)